# Well Placement and Control Optimization using a spatiotemporal proxy
### Misael M. Morales - 2024
***

In [171]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import os
from skimage.transform import resize
from sklearn.preprocessing import MinMaxScaler

from scipy.io import savemat, loadmat

import torch
import torch.nn as nn
from neuralop.models import *
from torch.utils.data import Dataset, DataLoader, random_split

sec2year   = 365.25 * 24 * 60 * 60
psi2pascal = 6894.76
co2_rho    = 686.5266
mega       = 1e6

n_timesteps = 33
nx, ny, nz  = 100, 100, 11

indexMap = loadmat('data_100_100_11/G_cells_indexMap.mat', simplify_cells=True)['gci']
Grid = np.zeros((nx,ny,nz)).flatten(order='F')
Grid[indexMap] = 1
Grid = Grid.reshape(nx,ny,nz, order='F')
tops = np.load('data_npy_100_100_11/tops_grid.npz')['tops']

In [173]:
def check_torch(verbose:bool=True):
    if torch.cuda.is_available():
        torch_version, cuda_avail = torch.__version__, torch.cuda.is_available()
        count, name = torch.cuda.device_count(), torch.cuda.get_device_name()
        if verbose:
            print('-'*60)
            print('----------------------- VERSION INFO -----------------------')
            print('Torch version: {} | Torch Built with CUDA? {}'.format(torch_version, cuda_avail))
            print('# Device(s) available: {}, Name(s): {}'.format(count, name))
            print('-'*60)
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        return device
    else:
        torch_version, cuda_avail = torch.__version__, torch.cuda.is_available()
        if verbose:
            print('-'*60)
            print('----------------------- VERSION INFO -----------------------')
            print('Torch version: {} | Torch Built with CUDA? {}'.format(torch_version, cuda_avail))
            print('-'*60)
        device = torch.device('cpu')
        return device

device = check_torch()

------------------------------------------------------------
----------------------- VERSION INFO -----------------------
Torch version: 2.3.1+cu121 | Torch Built with CUDA? True
# Device(s) available: 1, Name(s): NVIDIA GeForce RTX 3080
------------------------------------------------------------


In [182]:
class CustomDataset(Dataset):
    def __init__(self, data_folder:str='data_npy_100_100_11'):
        self.data_folder = data_folder
        
        self.x_folder = os.path.join(data_folder, 'inputs_rock_rates_locs_time')
        self.y_folder = os.path.join(data_folder, 'outputs_pressure_saturation')

        self.x_file_list = os.listdir(self.x_folder)
        self.y_file_list = os.listdir(self.y_folder)

    def __len__(self):
        return len(self.x_file_list)
    
    def __getitem__(self, idx):
        x  = np.load(os.path.join(self.x_folder, self.x_file_list[idx]))
        y  = np.load(os.path.join(self.y_folder, self.y_file_list[idx]))

        xm = np.concatenate([np.expand_dims(x['poro'],0), 
                             np.expand_dims(x['perm'],0), 
                             np.expand_dims(np.load('data_npy_100_100_11/tops_grid.npz')['tops'],0)], 
                             axis=0)
        
        xw = x['locs']
        xc = np.concatenate([np.zeros((1,xw.shape[-1])), x['ctrl']], axis=0)
        xt = np.insert(x['time'], 0, 0)
        yp = y['pressure']
        ys = y['saturation']

        return (xm, xw, xc, xt), (yp, ys)

In [ ]:
class FMiONet(nn.Module):
    def __init__(self, n_channels:int=100):
        super(FMiONet, self).__init__()
        self.n_ch = n_channels
        self.line1 = nn.Linear(5, self.n_ch)
        self.line2 = nn.Linear(2, self.n_ch)
        self.line3 = nn.Linear(1, self.n_ch)
        self.lift  = nn.Linear(270, 331)
        self.conv1 = nn.Conv3d(3, 8, kernel_size=3, padding=1)
        self.conv2 = nn.Conv3d(8, 16, kernel_size=3, padding=1)
        self.conv3 = nn.Conv3d(16, self.n_ch, kernel_size=3, padding=1)
        self.pool1 = nn.MaxPool3d(2)
        self.pool2 = nn.MaxPool3d((2,2,1))
        self.fno   = FNO1d(n_modes_height=4, n_layers=4, 
                           fno_block_precision='half', stabilizer='tanh', norm='instance_norm', 
                           in_channels=33, lifting_channels=64, hidden_channels=128, projection_channels=64, out_channels=33)
        
    def forward(self, x):
        xm, xw, xc, xt = x
        b, c, h, w, d = xm.shape
        zc = self.line1(xc)
        zw = self.line2(xw)
        zt = self.line3(xt)

        zm = self.pool1(self.conv1(xm))
        zm = self.pool1(self.conv2(zm))
        zm = self.pool2(self.conv3(zm))
        zm = zm.view(zm.shape[0], zm.shape[1], -1)
        
        b12 = torch.einsum('btc, bwc -> btwc', zc, zw).reshape(b, -1, self.n_ch)
        b123 = torch.einsum('bkc, bcp -> bkpc', b12, zm).reshape(b, -1, self.n_ch)
        btm = torch.einsum('btc, bkc -> btk', zt, b123).reshape(b, n_timesteps, 88, 270)
        
        zo = self.lift(btm).reshape(b, n_timesteps, -1)
        z = self.fno(zo)
        return z
    
model = FMiONet().to(device)
print('# params: {:,}'.format(sum(p.numel() for p in model.parameters() if p.requires_grad)))

In [183]:
dataset = CustomDataset()
trainset, testset  = random_split(dataset,  [1172, 100])
trainset, validset = random_split(trainset, [972,  200])

trainloader = DataLoader(trainset, batch_size=4, shuffle=True)
validloader = DataLoader(validset, batch_size=4, shuffle=False)
testloader  = DataLoader(testset, batch_size=4, shuffle=False)

In [184]:
for i, (x,y) in enumerate(trainloader):
    xm, xw, xc, xt = x
    yp, ys = y
    print(xm.shape)
    print(xw.shape)
    print(xc.shape)
    print(xt.shape)
    print('---')
    print(yp.shape)
    print(ys.shape)
    break

torch.Size([4, 3, 100, 100, 11])
torch.Size([4, 2, 5])
torch.Size([4, 34, 5])
torch.Size([4, 34])
---
torch.Size([4, 34, 29128])
torch.Size([4, 34, 29128])


***
# END